In [17]:
import json
import numpy as np
import matplotlib.pyplot as plt
from components_ import scene_creator, processor_creator
from lyd import utils, processing
from scipy.io import wavfile

def audio_entry(name):
    string = f'''<audio width="" height=" " src="docs/assets/{name}"  controls>'</audio>'''
    return string
from IPython.display import HTML, display

# Generate data

In [18]:
# Initialize objects
stimuli = scene_creator(2)
pL = processor_creator(fs=16e3)
pF = processor_creator(fs=16e3)
pS = processor_creator(fs=16e3)
pA = processor_creator(fs=16e3)

# Level adjuster
level_adjuster = processing.AdjustLevel(-23)

# Stimuli parameters
stimuli.update_param('signal_level',65)
stimuli.update_param('noise_type','Stationary')

# Initialize processes
pL.update_param('thr',100)
pL.update_param('ratio',1)
pL.update_param('wdrc','Fast Acting')

pF.update_param('thr',45)
pF.update_param('ratio',3)
pF.update_param('wdrc','Fast Acting')

pS.update_param('thr',45)
pS.update_param('ratio',3)
pS.update_param('wdrc','Slow Acting')

pA.update_param('thr',45)
pA.update_param('ratio',3)
pA.update_param('wdrc','Aware')

# Create dictionaries to iterate through
wdrc_dict = {
    'lin':pL,
    'fast':pF,
    'slow':pS,
    'aware':pA
}
rooms = {
    'Anechoic' : 'Anechoic',
    'Reverberant' : 'D'
}
snrs = {
    'inf' : 300,
    'high' : 16,
    'medium' : 6,
    'low' : 0
}
nrs = {
    'nr_off' : "None",
    'nr_on' : "Moderate"
}




# Initialize table
data = []
data.append(['Scene','Clean','Fast','Slow','Aware'])

# Iterate over noise reduction
for nr_key, nr_val in nrs.items():
    pL.update_param('nr',nr_val)
    pF.update_param('nr',nr_val)
    pS.update_param('nr',nr_val)
    pA.update_param('nr',nr_val)

    # Iterate over rooms
    for room_key, room_val in rooms.items():
        stimuli.update_param('room',room_val)
        
        # Iterate over snr
        for snr_key, snr_val in snrs.items():
            stimuli.update_param('snr',snr_val)
            
            # Generate stimuli
            stimuli.gen_scene()
            f,b,mix = stimuli.stim.transform()

            # Save input
            mix_adjusted = level_adjuster(mix)
            name = f"{room_key.lower()}-{snr_key}_snr.wav"
        
            
            row = []
            row.append(f"{room_key.lower()}-{snr_key}_snr-nr_{nr_val}")
            #row.append(audio_entry(name))

            # Iterate over compressors
            for wdrc_key, wdrc_val in wdrc_dict.items():
                wdrc_val.gen_sys()
                s = wdrc_val.s

                xc = s.transform(mix,b)
                xc_adjusted = level_adjuster(xc)[:,8000:]
                name = f"{room_key.lower()}-{snr_key}_snr-nr_{nr_key}-wdrc_{wdrc_key}.wav"
                wavfile.write("docs/assets/"+name,16000,xc_adjusted.T.astype(np.float32))
                row.append(audio_entry(name))

            name = f"{room_key.lower()}-{snr_key}_snr.wav"
            data.append(row)

        


100.0
45.0
45.0
45.0


# Audio 

### Properties
- Stationary noise (ICRA01)
- Rooms: 
    - Anechoic (Surrey Anechoic), or
    - Reverberant (Surrey Room D)
- Speech: TIMIT
- Speech level: 65 dB RMS
- SNR: 
    - inf, or
    - high: 16 dB SNR, or
    - medium: 6 dB SNR, or
    - low: 0 dB SNR
- Noise reduction
    - None, or
    - LogMMSE, 24dB maximum attenuation
- WDRC
    - Attack: 5ms
    - Release: 50ms (fast), 2000ms (slow)
    - Ratio: 3:1
    - Threshold: 45dB 
    - #Channels: 7




In [19]:
display(HTML(
   '<table class = "center"><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
))

Scene,Clean,Fast,Slow,Aware
anechoic-inf_snr-nr_None,',',','
anechoic-high_snr-nr_None,',',','
anechoic-medium_snr-nr_None,',',','
anechoic-low_snr-nr_None,',',','
reverberant-inf_snr-nr_None,',',','
reverberant-high_snr-nr_None,',',','
reverberant-medium_snr-nr_None,',',','
reverberant-low_snr-nr_None,',',','
anechoic-inf_snr-nr_Moderate,',',','
anechoic-high_snr-nr_Moderate,',',','


In [20]:
f = open("table.html","a")

f.write("<!DOCTYPE html>")
f.write("<html>")
f.write("<body>")

f.write(
    '<table class = "center"><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       ))

f.write("</html>")
f.write("</body>")

f.close()